In [1]:
#Import packages
import os
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import matplotlib.pyplot as plt
from pyproj import CRS

## Read in the data

In [2]:
#Filenames
DCFC_filename = 'Data\\NC_DCFC.shp'
graph_filename = 'Data\\NC_highways_all.graphml'

In [3]:
#Read DCFC points into geodataframe
print(f"Loading DCFC points from {DCFC_filename}")
gdf_DCFC = gpd.read_file(DCFC_filename)

Loading DCFC points from Data\NC_DCFC.shp


In [4]:
print(f"Loading graph from {graph_filename}")
nc_graph = ox.load_graphml(filename=os.path.basename(graph_filename),
                           folder=os.path.dirname(graph_filename))

Loading graph from Data\NC_highways_all.graphml


In [5]:
#Convert graph to a geodataframes
gdf_nodes, gdf_edges = ox.graph_to_gdfs(nc_graph)

In [ ]:
#Project to UTM (for analysis)
utm17N_prj = CRS.from_epsg(32617)
gdf_edges_utm = gdf_edges.to_crs(utm17N_prj)
gdf_DCFC_utm = gdf_DCFC.to_crs(utm17N_prj)

#Project to Web Mercator (for plotting)
wm_prj = CRS.from_epsg(3857)
gdf_edges_wm = gdf_edges.to_crs(wm_prj)
gdf_DCFC_wm = gdf_DCFC.to_crs(wm_prj)

## Visualize the data

In [ ]:
#Plot the data
ax = gdf_edges_wm.plot(figsize=(20,10))
gdf_DCFC_wm.plot(color='red',ax=ax)
ctx.add_basemap(ax)

## Identify "Safe Areas"
First, identify all "safe" areas, i.e., all alreas within a 100 mile drive of a DCFC charging location.

Source: https://github.com/gboeing/osmnx-examples/blob/master/notebooks/13-isolines-isochrones.ipynb

In [11]:
#Set the range
range_miles = 50

#Convert the distance to meters and halve 
range_meters = range_miles * 1609.344 / 2

#Create a list to hold the subgraphs created
subgraphs = []

#Iterate through each row and compute a subgraph
for i, row in gdf_DCFC.iterrows():
    #Status
    print('.',end='')
    #Get the coordinates
    thePoint = (row.geometry.y, row.geometry.x)
    #Get the ID
    theID = row.id
    #Get the nearest node
    theNode = ox.get_nearest_node(nc_graph,thePoint)
    #Get the subgraph
    subgraph = nx.ego_graph(G=nc_graph, n=theNode, radius=range_meters, distance='length')
    #Convert to a geodataframe
    subgdf = ox.graph_to_gdfs(G=subgraph,edges=True, nodes=False)
    #Add to list
    subgraphs.append(subgdf)
    
#Merge the subgraphs gdfs together
gdfSubgraphs = pd.concat(subgraphs)

....................................................................

In [15]:
#Save as a shapefile
gdfSubgraphs[['length','hwy','geometry']].to_file('Data\\Subgraphs.shp')

In [ ]:
#Plot the entire network
fig, ax = ox.plot_graph(nc_graph, fig_height=10,show=False, edge_color='k', edge_alpha=0.2, node_color='none',close=False)
#Add the subgraphs, in red
gdfSubgraphs.plot(ax=ax,color='red')
#Add the DCFC sites
gdf_DCFC.plot(ax=ax, color='blue')

## Next step:
The above figure reveals where a car with 100 mile range could drive. To increase this range, we'd need to add a charger anywhere within 50 miles of the existing "safe zone". To do that, we need to find all the terminal nodes 

In [76]:
#Get the coordinates
theRow = gdf_DCFC.iloc[6]
thePoint = (theRow.geometry.y, theRow.geometry.x)
#Get the ID
theID = row.id
#Get the nearest node
theNode = ox.get_nearest_node(nc_graph,thePoint)
#Get the subgraph
subgraph = nx.ego_graph(G=nc_graph, n=theNode, radius=range_meters, distance='length')

In [ ]:
the_map = ox.plot_graph_folium(nc_graph)
ox.plot_graph_folium(subgraph,graph_map=the_map)
the_map

In [ ]:
ax = subgraphs[6].to_crs(wm_prj).plot()
ctx.add_basemap(ax=ax)

In [17]:
gdfSubgraphs['hwy'] = gdfSubgraphs['highway'].apply(lambda x: x[0])
gdfSubgraphs[['length','hwy','geometry']].to_file('Data\\Subgraphs.shp')

In [25]:
ox.get_node(2706300546)

TypeError: 'int' object is not subscriptable

In [58]:
nx.edge_boundary(subgraph)

TypeError: edge_boundary() missing 1 required positional argument: 'nbunch1'

In [77]:
out_ids = []
out_nbrs = []
out_geoms = []
for n,data in subgraph.nodes(data=True):
    if subgraph.out_degree(n)==0 and nc_graph.out_degree(n)==0:
        out_ids.append(data['osmid'])
        out_geoms.append(Point(data['x'],data['y']))
        

In [78]:
gdfTerminals = gpd.GeoDataFrame()
gdfTerminals['osmid'] = out_ids
gdfTerminals['geometry'] = out_geoms
gdfTerminals.shape

(4, 2)

In [79]:
gdfTerminals.to_file('Data/TerminalPts.shp')

In [80]:
gNodes,gEdges = ox.graph_to_gdfs(nc_graph)

In [95]:
def fixIt(val):
    if type(val) == list:
        return val[0]
    else:
        return val

In [96]:
gEdges['hwy'] = gEdges.highway.apply(fixIt)

In [98]:
gEdges[['hwy','geometry']].to_file('Data/FixedEdges.shp')